<a href="https://colab.research.google.com/github/Rauloigs/Mercados-Financieros-P-Eficiente/blob/main/bl_csv_portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import blpor as blpor
# from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import altair as alt
import statsmodels.api as sm

### Obtener Retornos Equities & Market
- Un df de Equities
- Un Pandas Series de Market

In [ ]:
benchmark_returns = blpor.get_returns_spx()
asset_returns = blpor.get_returns_equities()
asset_returns.head(3)

,META,PLD,CVX,BA,ABT,AXP,DD,VZ,KO,NKE
2022-02-28,0.002610,-0.017670,0.025460,0.018977,-0.014731,0.004276,-0.009007,-0.008350,-0.009753,-0.016343
2022-03-01,-0.036383,-0.000617,0.038954,-0.052129,-0.020521,-0.088517,-0.022878,-0.001119,-0.004347,-0.032224
2022-03-02,0.022450,0.012815,0.029094,0.014769,0.018363,0.012557,0.019382,0.009838,0.007396,0.025832


### Matriz de Correlaciones

In [ ]:
corr = asset_returns.corr()
corr_matrix = corr.style.map(blpor.color_negative_red)
corr_matrix

,META,PLD,CVX,BA,ABT,AXP,DD,VZ,KO,NKE
META,1.000000,0.277062,0.099784,0.288545,0.259458,0.388813,0.327062,0.085503,0.116194,0.333437
PLD,0.277062,1.000000,0.239208,0.373036,0.510292,0.458777,0.439296,0.271757,0.399502,0.376912
CVX,0.099784,0.239208,1.000000,0.209482,0.147673,0.279765,0.339436,0.180485,0.164541,0.214231
BA,0.288545,0.373036,0.209482,1.000000,0.278705,0.438475,0.393980,0.098523,0.183689,0.360921
ABT,0.259458,0.510292,0.147673,0.278705,1.000000,0.356797,0.356296,0.286415,0.457240,0.363861
AXP,0.388813,0.458777,0.279765,0.438475,0.356797,1.000000,0.510399,0.186512,0.264920,0.395140
DD,0.327062,0.439296,0.339436,0.393980,0.356296,0.510399,1.000000,0.193199,0.321273,0.419762
VZ,0.085503,0.271757,0.180485,0.098523,0.286415,0.186512,0.193199,1.000000,0.348130,0.180413
KO,0.116194,0.399502,0.164541,0.183689,0.457240,0.264920,0.321273,0.348130,1.000000,0.343108
NKE,0.333437,0.376912,0.214231,0.360921,0.363861,0.395140,0.419762,0.180413,0.343108,1.000000


### Retornos
1. Retornos Promedio
2. Retornos CAPM

#### 1. Retornos Promedio
Obtenemos los retornos históricos de la serie.

In [ ]:
ex_rets_pue = blpor.annualize_rets(asset_returns, 252)
mean_rets = ex_rets_pue.to_frame(name='ER Mean') # to data frame
mean_rets

,ER Mean
META,0.328858
PLD,-0.101407
CVX,0.000644
BA,-0.111732
ABT,0.011755
AXP,0.107285
DD,-0.024057
VZ,-0.096470
KO,0.027584
NKE,-0.216525


#### 2. Retornos CAPM
- Betas:
  **(1) Betas Calculadas** Cálculo de acuerdo al SPX Index, últimos 3 años
  **(2)Betas Ajustadas Bloomberg** de acuerdo a la Terminal
- Risk Free Rate: Yield USGG3M Index (daily) de los últimos 3 años

A continuación observamos la diferencia entre las **Betas Calculadas** vs **Betas Ajustas de la Terminal - Bloomberg**

Existe una diferencia considerable (ver columna "Diferencia"), utilizaremos las Betas Ajustadas de Bloomberg

In [ ]:
betas_calculadas = blpor.get_betas(benchmark_returns, asset_returns)
betas_ajustadas_bloomberg = blpor.get_betas_bloomberg()
blpor.get_betas_bloomberg_vs_calculadas(betas_adj_bloomberg, betas_calculadas)

,Beta (Calculada SPX Index),Beta Adj Bloomberg,Diferencia
META,1.632025,1.080,-33.82%
PLD,1.027164,1.176,14.49%
CVX,0.557418,0.699,25.40%
BA,1.083497,1.031,-4.85%
ABT,0.640486,0.620,-3.20%
AXP,1.163123,1.106,-4.91%
DD,0.980971,0.867,-11.62%
VZ,0.318825,0.618,93.84%
KO,0.384988,0.456,18.45%
NKE,1.141602,0.818,-28.35%


Aquí obtenemos el **RF** a partir de **USGG3M Index** durante el mismo periodo de los Retornos de los Activos

In [ ]:
usgg3m_rf = blpor.usgg3m_risk_free_rate()
print(f"Risk-free rate: {usgg3m_rf:.5%}")

Risk-free rate: 4.29701%


Finalmente con las **Betas Ajustadas de Bloomberg** calculamos el **ER CAPM**

In [ ]:
capm_rets = blpor.rets_capm_rfmanual(betas_adj_bloomberg, benchmark_returns, risk_free_rate=usgg3m_rf, periods_per_year=252)
capm_rets

,ER CAPM
META,0.095906
PLD,0.100612
CVX,0.077232
BA,0.093505
ABT,0.073359
AXP,0.097181
DD,0.085466
VZ,0.073261
KO,0.065321
NKE,0.083064


Ahora comparamos los **Retornos Históricos vs Retornos CAPM**

In [ ]:
blpor.compare_mean_capm_returns(mean_rets, capm_rets)

alt.Chart(...)